## MEGAからワークフローと最終出力ファイルを復元する

In [ ]:
!pip install python-dotenv

import os
import subprocess
from dotenv import load_dotenv
from pathlib import Path

def run_cmd(cmd):
    print(f"$ {cmd}")
    result = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print("[stderr]", result.stderr)
    result.check_returncode()

# 1. .env からログイン情報を読み込む

load_dotenv(os.path.expanduser("~/.env"))
MEGA_EMAIL = os.getenv("MEGA_EMAIL")
MEGA_PASSWORD = os.getenv("MEGA_PASSWORD")

if not MEGA_EMAIL or not MEGA_PASSWORD:
    raise RuntimeError("環境変数 MEGA_EMAIL または MEGA_PASSWORD が設定されていません")

# 2. MEGA にログイン（subprocess 経由）

print("MEGA にログイン中...")
process = subprocess.run(
    ["mega-login", MEGA_EMAIL, MEGA_PASSWORD],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)
if process.returncode == 0:
    print("✅ MEGA ログイン成功")
else:
    print("❌ MEGA ログイン失敗")
    print(process.stderr)

# 3. ComfyUI/output から最新の .png ファイルを取得

output_path = Path("ComfyUI/output")
output_path.mkdir(parents=True, exist_ok=True)

print("ファイル一覧取得中（ComfyUI/output）")
result = subprocess.run("mega-ls /ComfyUI/output", shell=True, text=True, capture_output=True)
files = result.stdout.strip().split("\n") if result.stdout else []

png_files = [f for f in files if f.endswith(".png")]
if png_files:
    latest_png = sorted(png_files)[-1]  # 名前順で最新（タイムスタンプ取得は MEGAcmd では困難）
    print(f"最新ファイル: {latest_png}")
    run_cmd(f'mega-get "/ComfyUI/output/{latest_png}" "{output_path}"')
else:
    print("ComfyUI/output に .png ファイルが見つかりませんでした")

# 4. ComfyUI/user/default/workflows から .json をすべて取得

workflow_path = Path("ComfyUI/user/default/workflows")
workflow_path.mkdir(parents=True, exist_ok=True)

print("ファイル一覧取得中（ComfyUI/user/default/workflows）")
result = subprocess.run("mega-ls /ComfyUI/user/default/workflows", shell=True, text=True, capture_output=True)
files = result.stdout.strip().split("\n") if result.stdout else []

json_files = [f for f in files if f.endswith(".json")]
if json_files:
    for jf in json_files:
        print(f"ダウンロード中: {jf}")
        run_cmd(f'mega-get "/ComfyUI/user/default/workflows/{jf}" "{workflow_path}"')
else:
    print("ComfyUI/user/default/workflows に .json ファイルが見つかりませんでした")

# 5. ログアウト処理（安全のため明示）

#print("MEGAログアウト")
run_cmd("mega-logout")
